In [1]:
#!pip install --upgrade git+https://github.com/oKatanaaa/GCT

In [2]:
#!pip install --upgrade git+https://github.com/MakiResearchTeam/MakiFlow.git

In [3]:
#%tensorflow_version 1.x

In [1]:
import tensorflow as tf

In [2]:
from gct.tools import load_cifar10
import matplotlib.pyplot as plt
from makiflow.models.classificator import Classificator, CETrainer
from makiflow.generators.classification import cycle_generator
import tensorflow as tf
from makiflow.layers import *
import numpy as np
%matplotlib inline

In [3]:
(Xtrain, Ytrain), (Xtest, Ytest), (X_mean, X_std) = load_cifar10()

In [4]:
BATCH_SIZE = 64

In [5]:
def block(x, id, in_f, out_f):
    SX = x
    FX = x
    
    FX = BatchNormLayer(D=in_f, name=f'bn_main_{id}_1')(FX)
    FX = ActivationLayer(name=f'act_1_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=in_f, out_f=out_f, name=f'conv_main_{id}_1', activation=None)(FX)
    
    FX = BatchNormLayer(D=out_f, name=f'bn_main_{id}_2')(FX)
    FX = ActivationLayer(name=f'act_2_{id}')(FX)
    FX = ConvLayer(kw=3, kh=3, in_f=out_f, out_f=out_f, name=f'conv_main_{id}_2', activation=None)(FX)

    if in_f != out_f:
        SX = BatchNormLayer(D=in_f, name=f'bn_skip_{id}_1')(SX)
        SX = ConvLayer(kw=1, kh=1, in_f=in_f, out_f=out_f, name=f'conv_skip_{id}', activation=None)(SX)
    
    x_sum = SumLayer(name=f'sum_{id}')([FX, SX])
    return x_sum

In [6]:
# Create a model

In [7]:
inp = InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')
x = block(inp, id=1, in_f=3, out_f=32)
x = MaxPoolLayer(name='mp1')(x)
# 16x16
x = block(x, id=2, in_f=32, out_f=64)
x = block(x, id=3, in_f=64, out_f=64)
x = MaxPoolLayer(name='mp2')(x)
# 8x8
x = block(x, id=4, in_f=64, out_f=128)
x = block(x, id=5, in_f=128, out_f=128)
x = MaxPoolLayer(name='mp3')(x)
# 4x4
x = block(x, id=6, in_f=128, out_f=256)
x = block(x, id=7, in_f=256, out_f=256)
x = MaxPoolLayer(name='mp4')(x)
# 2x2
x = FlattenLayer('flat')(x)
# 4*256 = 1024
x = DenseLayer(in_d=1024, out_d=256, name='projection_head')(x)
x = DenseLayer(in_d=256, out_d=10, activation=None, name='classification_head')(x)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [8]:
model = Classificator(in_x=inp, out_x=x)
sess = tf.Session()
model.set_session(sess)

In [9]:
# Setup training

In [10]:
trainer = CETrainer(model=model, train_inputs=[InputLayer(input_shape=[BATCH_SIZE, 32, 32, 3], name='input')])

In [11]:
trainer.compile()



Loss is built.


In [12]:
opt = tf.train.AdamOptimizer()

In [13]:
gen = cycle_generator(Xtrain, Ytrain.reshape(-1), BATCH_SIZE)

In [32]:
info1 = trainer.fit_generator(gen, optimizer=opt, epochs=5, iter=100, print_period=10)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

New optimizer is used.


 10%|████████                                                                         | 10/100 [00:01<00:12,  7.31it/s]

Epoch: 0
Cross_Entropy: 5.1800777 
Training_Loss: 5.1800777 


Exception type: <class 'KeyboardInterrupt'>
Exception values: 
Exception tracer: <traceback object at 0x000002A1F5919948>


In [14]:
model.evaluate(Xtest, Ytest.reshape(-1))

157it [00:02, 59.14it/s] 


0.9001

In [ ]:
plt.plot(info1[CETrainer.TRAINING_LOSS])

In [15]:
np.random.randn(46, 48)[:48].shape

(46, 48)